## This COugh Model is base on the Freesound-1D CNN GlobalPool file

#### here I have transfered the trained weights from that model to retrain on cough or not cough dataset


In [1]:
import pickle
import os
import pandas as pd
import librosa
import librosa.display
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import tqdm.notebook as tqdm
from torchsummary import summary
import torch.optim as optim
import random
os.listdir('data/collected/')

['Annotations.csv', 'data']

In [2]:
df = pd.read_csv('data/collected/Annotations.csv')
df

,index,user_name,type_dir,directory,filename,times
0,1,sample_¦-¦ú++,quiet,cough,ZOOM0123_LR/Tr1/Tr2/Tr3.WAV,1.6 1.8 3.6 4.0 6.1 6.6 9.1 9.5 11.9 12.3 14.9...
1,2,sample_¦-¦ú++,quiet,continuous_cough,ZOOM0124_LR/Tr1/Tr2/Tr3.WAV,2.2 3.2 5.4 6.5 9.4 10.5 13.0 14.2 16.6 17.7 2...
2,3,sample_¦-¦ú++,quiet,throat_clear,ZOOM0125_LR/Tr1/Tr2/Tr3.WAV,1.8 2.2 4.2 4.6 7.2 7.6 10.0 10.4 12.9 13.3 15...
3,4,sample_¦-¦ú++,partitially_noisy,cough,ZOOM0120_LR/Tr1/Tr2/Tr3.WAV,1.2 1.5 3.2 3.5 5.8 6.2 8.3 8.6 10.9 11.2 13.3...
4,5,sample_¦-¦ú++,partitially_noisy,continuous_cough,ZOOM0121_LR/Tr1/Tr2/Tr3.WAV,1.2 2.1 3.9 4.8 7.3 8.3 10.6 11.6 14.4 15.5 18...
...,...,...,...,...,...,...
175,176,sample_++¦++e-¦,partitially_noisy,continuous_cough,ZOOM0028_LR/Tr1/Tr2/Tr3.WAV,1.6 2.4 4.6 5.7 9.2 10.5 13.8 14.5 18.1 18.8 2...
176,177,sample_++¦++e-¦,partitially_noisy,throat_clear,ZOOM0029_LR/Tr1/Tr2/Tr3.WAV,1.1 1.7 4.4 4.8 7.6 8.2 16.7 17.2 23.2 23.5 28...
177,178,sample_++¦++e-¦,noisy,cough,ZOOM0023_LR/Tr1/Tr2/Tr3.WAV,1.5 1.9 6.7 7.2 12.3 12.6 18.4 18.8 23.2 23.6 ...
178,179,sample_++¦++e-¦,noisy,continuous_cough,ZOOM0024_LR/Tr1/Tr2/Tr3.WAV,0.9 2.0 3.9 5.4 8.4 9.0 11.7 12.9 15.9 17.1 24...


In [3]:
[float(v) for v in list(df[0:1].times)[0].split(' ')[1::2]]

[1.8, 4.0, 6.6, 9.5, 12.3, 15.4, 18.7, 21.9, 24.8, 28.0]

In [3]:
base_path = 'data/collected/data/'
positives = {}
negatives = {}

for i in tqdm.tqdm(range(len(df))):
    
    user_name = list(df[i:i+1].user_name)[0]
    type_dir = list(df[i:i+1].type_dir)[0]
    directory = list(df[i:i+1].directory)[0]
    filename = list(df[i:i+1].filename)[0].split('_')[0]+'_'
    
    start_times = [float(v) for v in list(df[i:i+1].times)[0].strip().split(' ')[::2]]
    end_times = [float(v) for v in list(df[i:i+1].times)[0].strip().split(' ')[1::2]]
    
    dir_path = user_name +'/'+type_dir+'/'+directory+'/'+filename
    
    post_fixes = ['LR', 'Tr1', 'Tr2', 'Tr3']
    for post_fix in post_fixes:
        file_path = base_path + dir_path + post_fix + '.wav'
        
        raw_data = librosa.core.load(file_path, sr=None)
        audio_length = len(raw_data[0])/raw_data[1]
        
        target_sr = 11025
        resampled_data = librosa.core.resample(raw_data[0], raw_data[1], target_sr)
        
        last = 0.0
        for j, (st, et) in enumerate(zip(start_times, end_times)):
            negatives[dir_path + post_fix + '.wav/'+str(j)] = resampled_data[int(last*target_sr): int(st*target_sr)]
            positives[dir_path + post_fix + '.wav/'+str(j)] = resampled_data[int(st*target_sr): int(et*target_sr)]
            last = et

In [4]:
# save dictionary in pickle file
import pickle
output = open('data/collected/data/collected_data_11025.pkl', 'wb')
pickle.dump(positives, output)
output.close()

output = open('data/collected/data/collected_data_neg_11025.pkl', 'wb')
pickle.dump(negatives, output)
output.close()

In [5]:
Loaded_data = {}
if os.path.exists('data/freesound-audio-tagging/free_sound_11025.pkl'):
    Loaded_data = pickle.load(open('data/freesound-audio-tagging/free_sound_11025.pkl', 'rb'))
    
def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data-min_data)/(max_data-min_data+1e-6)
    return data-0.5

def load_audio_file(file_path, input_length=4096):
    
    if file_path not in Loaded_data:
        data = librosa.core.load(file_path, sr=None) 
        data = librosa.core.resample(data[0], data[1], 11025)
        Loaded_data[file_path] = data
    else:
        data = Loaded_data[file_path]
    
    if len(data)>input_length:
        max_offset = len(data)-input_length
        offset = np.random.randint(max_offset)
        data = data[offset:input_length+offset]
        
    else:
        T = np.zeros(input_length, dtype=float)
        T[:len(data)] = data
        data = T
        #max_offset = input_length - len(data)
        ##offset = np.random.randint(max_offset)
        #data = np.pad(data, (offset, input_length - len(data) - offset), "constant")
        
    data = audio_norm(data)
    return np.array([data])

In [6]:
class CoughDataset(torch.utils.data.Dataset):
    def __init__(self, cough_data_path, non_cough_data_path, train=True, split = 0.8, seed=1):
        
        # Load Data
        self.input_length = 4096
        self.cough_data = pickle.load(open(cough_data_path, 'rb'))
        self.keys = list(self.cough_data.keys())
        self.non_cough_df = pd.read_csv(non_cough_data_path)
        
        # set Seed
        random.seed(seed)
        
        # shuffle Data
        random.shuffle(self.keys)
        
        
        self.non_cough_data = pd.read_csv(non_cough_data_path)
        self.non_cough_data = self.non_cough_data[self.non_cough_data.label!='Cough'][:len(self.keys)]
        
        
        # split Data
        if train:
            self.keys = self.keys[:int(len(self.keys)*split)]
            self.non_cough_data = self.non_cough_data[:int(len(self.keys)*split)]
        else:
            self.keys = self.keys[int(len(self.keys)*split):]
            self.non_cough_data = self.non_cough_data[int(len(self.keys)*split):]
        

        self.non_cough_idx = 0
        self.cough_idx = 0
        self.max_len = len(self.keys)
        # load env_sounds
        
        
        
    
    def __len__(self):
        return len(self.keys)*2
    
    def __getitem__(self, idx):
        
        # rnadomly select positive of negetaive instannce
        if random.choice([False, True]):
            non_cough_file_path = 'data/freesound-audio-tagging/audio_train/' + list(self.non_cough_df[self.non_cough_idx: self.non_cough_idx+1].fname)[0]
            self.non_cough_idx = 0 if self.non_cough_idx+1>=self.max_len else self.non_cough_idx+1
            return load_audio_file(non_cough_file_path), 0
        
        else:
            cough_data_base = np.zeros(self.input_length)
            cough_data_instance = self.cough_data[self.keys[self.cough_idx]]
            self.cough_idx = 0 if self.cough_idx+1>=self.max_len else self.cough_idx+1
            
            instance_length = len(cough_data_instance)
            
            if instance_length/self.input_length < 1:
                start_idx =  int(self.input_length/2 - int(instance_length/2))
                cough_data_base[start_idx:instance_length+start_idx] = cough_data_instance
            
            else:
                start_idx = int(instance_length/2)-int(self.input_length/2)
                cough_data_base = cough_data_instance[start_idx: start_idx+self.input_length]
            
            return np.array([audio_norm(cough_data_base)]), 1
                
            

In [7]:
Cough_Train_Data = CoughDataset('data/collected/data/collected_data_11025.pkl', 'data/freesound-audio-tagging/train.csv')
Cough_Test_Data = CoughDataset('data/collected/data/collected_data_11025.pkl', 'data/freesound-audio-tagging/train.csv', train = False)

mini_batch_size = 64
Cough_Train_Dataloader = DataLoader(Cough_Train_Data, batch_size=mini_batch_size, shuffle=True)
Cough_Test_Dataloader = DataLoader(Cough_Test_Data, batch_size=mini_batch_size, shuffle=True)

In [10]:
class FreeSound_Sense(torch.nn.Module):
    
    def __init__(self):
        super(FreeSound_Sense, self).__init__()
        self.conv1d_1_16_9 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=9, padding=True)
        self.conv1d_16_16_9 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=9, padding=True)
        self.conv1d_16_32_3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=True)
        self.conv1d_32_32_3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=True)
        self.conv1d_32_96_3 = nn.Conv1d(in_channels=32, out_channels=96, kernel_size=3, padding=True)
        self.conv1d_96_96_3 = nn.Conv1d(in_channels=96, out_channels=96, kernel_size=3, padding=True)
        
        self.maxpool_16 = nn.MaxPool1d(16)
        self.maxpool_4 = nn.MaxPool1d(4)
        
        self.relu = nn.ReLU()
        self.sigm = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(0.1)
        
        self.fc_96_64 = nn.Linear(in_features=96, out_features=64)
        self.fc_64_512 = nn.Linear(in_features=64, out_features=512)
        self.fc_512_42 = nn.Linear(in_features=512, out_features=42)
        
        
    def forward(self, x):
        
        # First Block
        x = self.conv1d_1_16_9(x)
        x = self.relu(x)
        x = self.conv1d_16_16_9(x)
        x = self.relu(x)
        x = self.maxpool_16(x)
        x = self.dropout(x)
        
        # Second Block
        x = self.conv1d_16_32_3(x)
        x = self.relu(x)
        x = self.conv1d_32_32_3(x)
        x = self.relu(x)
        x = self.maxpool_4(x)
        x = self.dropout(x)
        
        # Third Block
        x = self.conv1d_32_32_3(x)
        x = self.relu(x)
        x = self.conv1d_32_32_3(x)
        x = self.relu(x)
        x = self.maxpool_4(x)
        x = self.dropout(x)
        
        # Fourth Block
        x = self.conv1d_32_96_3(x)
        x = self.relu(x)
        x = self.conv1d_96_96_3(x)
        x = self.relu(x)
        x = self.maxpool_4(x)
        x = torch.mean(x, 2)
 
        # Final Layers
        x = torch.flatten(x, start_dim=1)
        x = self.fc_96_64(x)
        x = self.relu(x)
        x = self.fc_64_512(x)
        x = self.relu(x)
        x = self.fc_512_42(x)
        x = self.softmax(x)
        
        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model = FreeSound_Sense()
Model.float()
Model.to(device)
summary(Model, (1, 4096))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 16, 4090]             160
              ReLU-2             [-1, 16, 4090]               0
            Conv1d-3             [-1, 16, 4084]           2,320
              ReLU-4             [-1, 16, 4084]               0
         MaxPool1d-5              [-1, 16, 255]               0
           Dropout-6              [-1, 16, 255]               0
            Conv1d-7              [-1, 32, 255]           1,568
              ReLU-8              [-1, 32, 255]               0
            Conv1d-9              [-1, 32, 255]           3,104
             ReLU-10              [-1, 32, 255]               0
        MaxPool1d-11               [-1, 32, 63]               0
          Dropout-12               [-1, 32, 63]               0
           Conv1d-13               [-1, 32, 63]           3,104
             ReLU-14               [-1,

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Model = FreeSound_Sense()
# uncomment if doing transfer learning
Model.load_state_dict(torch.load("model_weights/FreeSound_1D_conv_small_164_epoch.stDict"))
Model.fc_512_42 = nn.Linear(in_features=512, out_features=2)

# Comment tnext line of doig  tarnsfer learning
#Model.load_state_dict(torch.load("model_weights/Cough_Data_1D_conv_global_pool_deep_batchnorm_15_epoch_95Acc.stDict"))



Model.float()
Model.to(device)

# Change the last layer
summary(Model, (1, 4096))

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(Model.parameters(), lr=0.001, momentum=0.9)

# For the first 100 epoch only train the last 3 layers of FC after that connect the below loop
for i, param in enumerate(Model.parameters()):
    if i not in [15, 16, 17]:
        param.requires_grad = False     
optimizer = optim.Adam(Model.parameters(), lr=0.0005, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 16, 4090]             160
              ReLU-2             [-1, 16, 4090]               0
            Conv1d-3             [-1, 16, 4084]           2,320
              ReLU-4             [-1, 16, 4084]               0
         MaxPool1d-5              [-1, 16, 255]               0
           Dropout-6              [-1, 16, 255]               0
            Conv1d-7              [-1, 32, 255]           1,568
              ReLU-8              [-1, 32, 255]               0
            Conv1d-9              [-1, 32, 255]           3,104
             ReLU-10              [-1, 32, 255]               0
        MaxPool1d-11               [-1, 32, 63]               0
          Dropout-12               [-1, 32, 63]               0
           Conv1d-13               [-1, 32, 63]           3,104
             ReLU-14               [-1,

In [14]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
Model.to(device)

epoch_progress_bar = tqdm.tqdm(range(0, 500))

for epoch in epoch_progress_bar:
    avg_epoch_loss = 0
    data_progress_bar = tqdm.tqdm(Cough_Train_Dataloader)
    positives=0
    for data, targets in data_progress_bar:
        data = data.float().to(device)
        targets = targets.long().to(device)


        optimizer.zero_grad()
        outputs = Model(data)
        

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        loss_val = loss.item()
        avg_epoch_loss+=loss_val
        data_progress_bar.set_description(desc="Loss: "+str(loss_val))
        
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        targets = targets.cpu().numpy()
        positives += np.sum(targets==outputs)
    
    print('Epoch Loss: ', str(avg_epoch_loss/len(Cough_Train_Dataloader)))
    print('Train Acc ', str(positives*100/(len(Cough_Train_Dataloader)*mini_batch_size)))
    
    # Validation
    data_test_progress_bar = tqdm.tqdm(Cough_Test_Dataloader)
    positives=0
    for data, targets in data_test_progress_bar:
        data = data.float().to(device)
        targets = targets.numpy()
        outputs = Model(data)
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        positives += np.sum(targets==outputs)
        
    print('Valid Acc ', str(positives*100/(len(Cough_Test_Dataloader)*mini_batch_size)))


Epoch Loss:  0.5185574144124985
Train Acc  79.0264423076923



Valid Acc  78.80434782608695



Epoch Loss:  0.49667586091455523
Train Acc  80.73489010989012



Valid Acc  78.94021739130434



Epoch Loss:  0.4988730702098909
Train Acc  80.46016483516483



Valid Acc  79.38179347826087


KeyboardInterrupt: 

In [15]:
torch.save(Model.state_dict(), "model_weights/Cough_Data_1D_conv_small_3_epoch_80Acc.stDict")

In [146]:
i=0
for data in Cough_Train_Dataloader:
    if i==2:
        O = Model(data[0].float().to(device))
        A = data[1]
        #print(data[1])
        break
    i+=1
O = O.detach().cpu().numpy()
K = np.argmax(O, axis=1)==A.numpy()
print(np.sum(K)/len(K), np.sum(K))
list(zip(A, np.argmax(O, axis=1), K))

0.984375 63


[(tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(0), 1, False),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(0), 0, True),
 (tensor(1), 1, True),
 (tensor(0), 0, True),
 (tensor(0